<a href="https://colab.research.google.com/github/ducndh/cs320MachineLearningProjects/blob/main/PredictProfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
#force_remount = True means that if you run this cell again, it will re-load your
#drive. That's necessary because otherwise, if you make changes to files in Drive,
#you won't see them in Colab - you have to re-run this cell to load the changes.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Check if accessing correctly
!ls drive/MyDrive/Colab\ Notebooks

'drive/MyDrive/Colab Notebooks/TestTable.xlsx'


In [ ]:
import pandas as pd
df = pd.read_excel("drive/MyDrive/Colab Notebooks/TestTable.xlsx",engine='openpyxl')
df = df.dropna(thresh=df.shape[0]*0.1,axis=1).reset_index(drop=True)
df = df.dropna(thresh=df.shape[1]*0.1,axis=0).reset_index(drop=True)
df['Percentage_change'] = df['Price']/df['Ban']
df['Value_change'] = df['Price'] - df['Ban']
df.drop(df.columns[[0, 1, 2, 3, 4]], axis = 1, inplace = True)
df.drop(['COGS'], axis = 1, inplace = True)
# df['COGS'] = df['COGS'].fillna(0)
df

,Volume,Market Capitalization,Reported Earnings,Sales,Cash,Investments,Receivable,Inventory,Debts,Percentage_change,Value_change
0,1.708000e+08,2.954840e+12,1.802605e+11,8.011573e+12,6.454748e+11,7.209065e+11,1.612785e+12,8.627492e+11,3.206103e+12,1.341085,4400.0
1,1.493616e+07,2.210552e+11,9.591231e+09,2.182639e+11,1.109879e+10,7.225952e+10,3.110992e+10,6.518940e+10,1.524498e+10,1.286957,3300.0
2,1.370721e+07,5.482883e+11,6.672898e+10,3.959384e+11,3.033181e+10,6.021715e+10,4.882141e+10,1.226902e+11,1.107132e+11,1.212121,7000.0
3,9.600000e+06,2.083200e+11,3.091018e+10,3.804570e+11,5.511212e+10,0.000000e+00,8.327268e+10,1.493723e+11,1.255208e+11,1.112821,2200.0
4,2.239968e+07,7.951885e+11,2.304002e+11,1.688854e+12,7.968336e+10,0.000000e+00,3.634851e+11,4.622277e+11,7.846145e+11,1.908602,16900.0
...,...,...,...,...,...,...,...,...,...,...,...
2909,7.199652e+06,1.022351e+11,1.195425e+10,1.644646e+11,4.693853e+08,4.730000e+08,2.261017e+10,4.508860e+10,4.640118e+10,2.366667,8200.0
2910,7.268568e+06,5.160683e+11,6.506780e+10,2.347075e+11,4.771054e+10,8.218226e+10,6.964251e+10,2.605016e+09,8.125837e+10,1.272401,15200.0
2911,1.255644e+08,3.729262e+12,2.568414e+11,3.701615e+11,2.287835e+11,5.106853e+11,7.055760e+10,2.336182e+10,1.231556e+11,1.129278,3400.0
2912,7.204564e+06,2.139755e+11,2.416730e+10,5.123238e+11,1.901906e+10,9.120300e+10,1.258465e+11,3.881369e+10,1.165270e+11,2.216418,16300.0


In [ ]:
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, LeakyReLU
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
tf.keras.backend.set_floatx('float64')

def plot_performance(history, metrics = ["loss", "my_accuracy"]):
  for metric in metrics:
    plt.plot([i-.5 for i in range(1,epochs+1)], history.history[metric], label="Train " + metric)
    plt.plot(range(1,epochs+1), history.history["val_" + metric], label="Validation " + metric)
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.title(metric + " over time")
    plt.show()

def create_model():
  model = keras.Sequential()
  model.add(Dense(1024, name = "hidden_layer_1", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
    bias_regularizer=regularizers.l2(1e-3)))
  model.add(LeakyReLU(0.3))
  model.add(Dropout(0.1))
  model.add(Dense(256, name = "hidden_layer_2", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
    bias_regularizer=regularizers.l2(1e-3)))
  model.add(LeakyReLU(0.3))
  model.add(Dropout(0.1))
  model.add(Dense(1, name = "output_layer"))
  model.compile(optimizer = 'adam', 
                loss = 'mean_absolute_error',
                metrics=[my_accuracy])
  return model

accuracy = []
for i in range(5):
  epochs= 50
  model = create_model()
  callbacks = [EarlyStopping(monitor='val_my_accuracy', patience=epochs),
          ModelCheckpoint(filepath='best_model.h5', monitor='val_my_accuracy', save_best_only=True)]
  history_model = model.fit(X, y, callbacks = callbacks, epochs=epochs, validation_split=0.1)

  plot_performance(history_model)
  accuracy.append(model.evaluate(X_test, y_test)[1])
  # for layer in model.layers: 
  #   print(layer.get_config(), layer.get_weights())
print(np.mean(accuracy), max(accuracy))